In [116]:
# librairies de gestion des chemins d'accès des fichiers/dossiers
from pathlib import Path

# librairies de manipulation des données tabulaires
import pandas as pd
import numpy as np

# librairies de manipulation des données textuelles
import nltk
import gensim
import re
import string
from bs4 import BeautifulSoup
from textblob import TextBlob
import contractions

# librairies de visualtion
import IPython.display
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline

# module de chargement des données et fonctions utiles
import FETCH_LOAD_DATAS
import ACP_functions

In [2]:
%%time
raw_queries = pd.read_pickle('datasets/raw_queries.pkl')

CPU times: total: 3.72 s
Wall time: 8.97 s


In [59]:
%%time
df = raw_queries.copy()
df.sort_values('creation_date', inplace=True)
df.reset_index(drop=True, inplace=True)

CPU times: total: 1.48 s
Wall time: 1.49 s


In [60]:
df

,question_id,creation_date,title,body,tags,score,view_count,answer_count
0,59549239,2020-01-01 00:07:37.293,Modifying Bonita Process Instance after Started,<p>Is it possible to change the processinstanc...,bonita,1,63,1
1,59549244,2020-01-01 00:09:53.770,What does Elevation in DrawShadow do exactly?,"<p>I know that <a href=""https://github.com/flu...",c++|flutter|dart|skia,3,380,1
2,59549281,2020-01-01 00:23:15.297,How to use a trait object that contains a meth...,<p>What's the correct way to use a trait objec...,rust,3,952,1
3,59549286,2020-01-01 00:24:49.650,A way for users to store selections as inputs ...,<p>I'm trying to create a way in which a user ...,r|shiny|shiny-server|shiny-reactivity|shinyapps,2,78,1
4,59549303,2020-01-01 00:30:49.253,How to open downloaded photo using gallery app...,"<p>I am facing a problem, cannot figure out ho...",react-native|linker|expo|gallery,5,3235,1
...,...,...,...,...,...,...,...,...
999995,73842028,2022-09-25 04:18:10.303,Can you do a $pull in a MongoDB Bulk Write whi...,<p>Version: MongoDB v 4.2.18.</p>\n<p>I have t...,mongodb,1,14,1
999996,73842042,2022-09-25 04:23:21.587,non static method can't be called statically,<p>I am so lost with how to get this working. ...,php|laravel,1,22,1
999997,73842118,2022-09-25 04:50:21.720,Sorting os.listdir()'s arbitrary order for des...,"<p><a href=""https://i.stack.imgur.com/kM7UP.pn...",python|list|sorting,1,18,1
999998,73842153,2022-09-25 05:03:58.270,Big O complexity of nested loop,<p>What could be the big O of this code?</p>\n...,big-o,1,13,1


# Pré-traitement des données et feature engineering

Avant d'aborder la problématique du projet de manière non supervisée puis supervisée, nous allons réaliser plusieurs pré-traitements des données qui nous serviront à comparer les résultats avec chacun des approches, puis nous comparerons les résultats obtenus dans les meilleures configurations de chaque approche et nous déciderons du modèle final que nous déploierons sous forme d'API afin qu'il puisse être utilisé par les utilisateurs novices de **StackOverFlow**.

**NB** : Dans un premier temps, nous allons conserver tous les posts de notre jeu de données, et nous verrons dans un second temps si nous pourrons améliorer les performances de nos modèles en ne conservant que les posts avec un certain niveau de score ou nombre de réponses ou de vues ou une combinaison de tous ces critères à la fois.

In [61]:
# df_datas = df[['title', 'body']].copy()
# df_target = df['tags'].copy()

## Conservation des paragraphes des corps des posts uniquement

In [111]:
def keep_text_from_tags(html, tags=['p']):
    """
        But :
        
        Analyse le contenu HTML et extrait le contenu textuel des balises passées en argument.
        
        Arguments :
        
        htmt : document au format HTML.
        tags : list des balises dont l'on veut extraire le contenu textuel.
        
        Valeur retournée :
        
        Une chaîne de caractère issue de la jointure des contenus textuels des différentes balises analysées.
    """
    
    soup = BeautifulSoup(html, "lxml")
    
    list_text = list()
    
    for tag in tags:
        list_text.extend([x.get_text() for x in soup.find_all(tag)])
 
    
    return ' '.join(list_text)

In [112]:
# df_datas['body'] = df_datas['body'].apply(keep_text_from_tags)

In [ ]:
%store df_datas

## Nettoyage du texte

Cette étape est constituée des tâches suivantes :

   1. **Formattage** : Passsage en minuscule de tout le texte (dans un second temps, nous pourrons conserver la trace des mots comprennant des majuscules pour voir si cela améliore les performances de nos modèles)
   2. **Othographe** : Corriger les fautes d'orthographes ainsi que les abréviations argotiques
   3. **Filtre syntaxique** : Tri des mots et conservation des noms uniquement (POS_tagging)
   4. **Suppression du bruit** : Suppression des caractères spéciaux, ponctuations, caractères non-ASCII, URL, balises HTML...
   

In [120]:
# 1. FORMATTAGE

# df_datas = df_datas.apply(lambda x: x.str.lower())

In [132]:
# 2. ORTHOGRAPHE

# df_datas['title'] = df_datas['title'].apply(lambda x: contractions.fix(x))
# df_datas['body'] = df_datas['body'].apply(lambda x: contractions.fix(x))

df_datas['title'] = df_datas['title'].apply(lambda x: TextBlob(x).correct())
df_datas['body'] = df_datas['body'].apply(lambda x: TextBlob(x).correct())

KeyboardInterrupt: 

In [134]:
df_datas.to_pickle('datasets/tests/df_datas_contractions.pkl')